In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Is CUDA available? {torch.cuda.is_available()}")

print(f"Selected device: {device}")
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    
    # Name of the currently active GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    
    # Check current memory usage (optional, useful in a running process)
    # print(f"Allocated memory: {torch.cuda.memory_allocated(0)} bytes")
else:
    print("CUDA not available. Check GPU drivers and PyTorch installation.")

Is CUDA available? True
Selected device: cuda
Number of GPUs: 1
GPU Name: NVIDIA RTX 2000 Ada Generation Laptop GPU


In [ ]:
import os
from ultralytics import YOLO
from torch.utils.data import DataLoader, TensorDataset
import torch


# X_train = torch.tensor(list(train.values()), dtype=torch.float32) # Adjust dtype as needed
# y_train = torch.tensor(list(train.keys()), dtype=torch.long) # Use torch.long for classification labels/indices
#
# X_val = torch.tensor(list(val.values()), dtype=torch.float32)
# y_val = torch.tensor(list(val.keys()), dtype=torch.long)
#
# X_test = torch.tensor(list(test.values()), dtype=torch.float32)
# y_test = torch.tensor(list(test.keys()), dtype=torch.long)

### Creating Dataset
# print("X_train",X_train[:2])
# print("y_train",y_train[:2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Check 2: See what device was selected
print(f"Selected device: {device}")
if torch.cuda.is_available():
    # Number of GPUs detected
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    
    # Name of the currently active GPU
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    
    # Check current memory usage (optional, useful in a running process)
    # print(f"Allocated memory: {torch.cuda.memory_allocated(0)} bytes")
else:
    print("CUDA not available. Check GPU drivers and PyTorch installation.")
# --- Configuration des chemins ---
# Assurez-vous que ces chemins sont corrects par rapport à l'emplacement où vous exécutez le script
DATA_YML_PATH = 'C:\\Users\\boume\\Multimodal-Diagnosis-by-AI\\part_Two\\DetectionTumeur\\data.yaml'       # Chemin vers le fichier SANS augmentation
DATA2_YML_PATH = 'C:\\Users\\boume\\Multimodal-Diagnosis-by-AI\\part_Two\\DetectionTumeur\\data2.yaml'      # Chemin vers le fichier AVEC augmentation (identique à data.yaml)
HYP_NO_AUG_PATH = 'C:\\Users\\boume\\Multimodal-Diagnosis-by-AI\\part_Two\\DetectionTumeur\\hyp_no_aug.yaml' # Chemin vers le fichier d'hyperparamètres SANS augmentation
# YOLOv8 utilise automatiquement ses hyperparamètres par défaut s'ils ne sont pas spécifiés.

# --- Initialisation du modèle de base ---
# Nous utilisons 'yolov8n.pt' (nano) comme point de départ pour le transfer learning.
# Remplacez par 'yolov8s.pt' (small) ou autre si vous souhaitez un modèle plus grand.
MODEL_WEIGHTS = 'yolov8n.pt'
EPOCHS = 50 # Nombre d'époques pour l'entraînement (ajustez selon vos besoins)

# -------------------------------------------------------------
# SCÉNARIO 1 : Entraînement SANS Augmentation (Utilise data.yaml et hyp_no_aug.yaml)
# -------------------------------------------------------------

print("\n🚀 Démarrage de l'entraînement SANS augmentation (data.yaml)...")

# Charger un modèle YOLOv8
model_no_aug = YOLO(MODEL_WEIGHTS)

# Lancer l'entraînement
# NOTE : Dans YOLOv8, le fichier hyp.yaml est spécifié via l'argument 'cfg'
# qui remplace le fichier de configuration du modèle (mais il contient aussi les hyp).
# L'argument 'augment=False' est une manière plus simple d'assurer que la mosaïque, etc. soit désactivée.
results_no_aug = model_no_aug.train(
    data=DATA_YML_PATH,
    epochs=EPOCHS,
    imgsz=640,                 # Taille de l'image (standard)
    batch=-1,                  # Utiliser le batch size optimal pour votre GPU
    name='yolo_no_aug_run',    # Nom du dossier de sortie (runs/detect/yolo_no_aug_run)
    # L'argument 'augment' est très utile pour simplifier la désactivation dans YOLOv8
    # 'augment=False' désactive les augmentations géométriques (mosaïque, mixup, etc.)
    augment=False,
    # Vous pouvez également pointer vers votre fichier hyp_no_aug.yaml si vous avez modifié
    # d'autres hyperparamètres non liés à l'augmentation :
    # cfg=HYP_NO_AUG_PATH
)

print("\n✅ Entraînement SANS augmentation terminé. Modèle sauvegardé dans runs/detect/yolo_no_aug_run")

# -------------------------------------------------------------
# SCÉNARIO 2 : Entraînement AVEC Augmentation (Utilise data2.yaml et hyperparamètres par défaut)
# -------------------------------------------------------------

print("\n🚀 Démarrage de l'entraînement AVEC augmentation (data2.yaml)...")

# Recharger le modèle de base pour un entraînement propre
model_with_aug = YOLO(MODEL_WEIGHTS)

# Lancer l'entraînement
# L'omission des arguments 'augment=False' permet d'utiliser les augmentations par défaut de YOLOv8.
results_with_aug = model_with_aug.train(
    data=DATA2_YML_PATH,
    epochs=EPOCHS,
    imgsz=640,
    batch=-1,
    name='yolo_with_aug_run', # Nom du dossier de sortie (runs/detect/yolo_with_aug_run)
    # L'argument 'augment' n'est pas spécifié, donc il utilise True par défaut.
)

print("\n✅ Entraînement AVEC augmentation terminé. Modèle sauvegardé dans runs/detect/yolo_with_aug_run")

# --- Évaluation du modèle SANS augmentation sur l'ensemble de test ---
print("\n🔍 Évaluation du modèle SANS augmentation...")
model_no_aug.val(
    data=DATA_YML_PATH,
    split='test',
    # Le chemin du modèle est automatiquement mis à jour après l'entraînement
)

# --- Évaluation du modèle AVEC augmentation sur l'ensemble de test ---
print("\n🔍 Évaluation du modèle AVEC augmentation...")
model_with_aug.val(
    data=DATA2_YML_PATH,
    split='test',
    # Le chemin du modèle est automatiquement mis à jour après l'entraînement
)

print("\n✨ Évaluation terminée. Consultez les résultats dans les dossiers de run.")

Is CUDA available? True
Selected device: cuda
Number of GPUs: 1
GPU Name: NVIDIA RTX 2000 Ada Generation Laptop GPU

🚀 Démarrage de l'entraînement SANS augmentation (data.yaml)...
Ultralytics 8.3.221  Python-3.11.0 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX 2000 Ada Generation Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\boume\Multimodal-Diagnosis-by-AI\part_Two\DetectionTumeur\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, 